In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import czifile as zis
from apeer_ometiff_library import io, processing, omexmlClass
import os
from matplotlib import pyplot as plt, cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from skimage.viewer import ImageViewer
import skimage.io
import matplotlib.colors as colors
import numpy as np

In [2]:
#basefolder = r'/datadisk1/tuxedo/IPython_Notebooks/testdata'
basefolder = r'/home/sebi06/testdata'

#filename_ometiff = os.path.join(basefolder, 'Filter_with_Particles_big.ome.tiff')
#filename_ometiff = os.path.join(basefolder, 'S=2_T=5_CH=3_CH=2_A2.ome.tiff')
filename_ometiff = os.path.join(basefolder, 'Osteosarcoma_01.ome.tiff')
#filename_czi = os.path.join(basefolder, 'Filter_with_Particles_big.czi')
#filename_czi = os.path.join(basefolder, '8Brains_DAPI_5X_stitched.czi')
#filename_czi = os.path.join(basefolder, 'S=2_T=5_Z=3_CH=2_A2.czi')

In [3]:
def create_metadata_dict():
    """
    A Python dictionary will be created to hold the relevant metadata.
    """

    metadata = {'Directory': '',
                'Filename': '',
                'Name': '',
                'AcqDate': '',
                'TotalSeries': 0,
                'SizeX': 0,
                'SizeY': 0,
                'SizeZ': 0,
                'SizeC': 0,
                'SizeT': 0,
                'DimOrder BF': 'n.a.',
                'NA': 0,
                'ObjMag': 0,
                'ObjID': 'n.a.',
                'XScale': 0,
                'YScale': 0,
                'ZScale': 0,
                'XScaleUnit': 'n.a.',
                'YScaleUnit': 'n.a.',
                'ZScaleUnit': 'n.a.',
                'DetectorModel': [],
                'DetectorName': [],
                'DetectorID': None,
                'InstrumentID': None,
                'Channels': [],
                'ImageIDs': []
               }

    return metadata

def get_metadata(filename, omexml,
                 series=0):
    
    # create dictionary for metadata and get OME-XML data
    metadata = create_metadata_dict()
    md = omexmlClass.OMEXML(omexml)
    
    # get directory and filename etc.
    metadata['Directory'] = os.path.dirname(filename)
    metadata['Filename'] = os.path.basename(filename)
    metadata['AcqDate'] = md.image(series).AcquisitionDate
    metadata['Name'] = md.image(series).Name
    
    # get image dimensions
    metadata['SizeT'] = md.image(series).Pixels.SizeT
    metadata['SizeZ'] = md.image(series).Pixels.SizeZ
    metadata['SizeC'] = md.image(series).Pixels.SizeC
    metadata['SizeX'] = md.image(series).Pixels.SizeX
    metadata['SizeY'] = md.image(series).Pixels.SizeY
    # get number of series
    metadata['TotalSeries'] = md.get_image_count()
    
    # get dimension order
    metadata['DimOrder BF'] = md.image(series).Pixels.DimensionOrder
    
    # get the scaling
    metadata['XScale'] = md.image(series).Pixels.PhysicalSizeX
    metadata['XScaleUnit'] = md.image(series).Pixels.PhysicalSizeXUnit
    metadata['YScale'] = md.image(series).Pixels.PhysicalSizeY
    metadata['YScaleUnit'] = md.image(series).Pixels.PhysicalSizeYUnit
    metadata['ZScale'] = md.image(series).Pixels.PhysicalSizeZ
    metadata['ZScaleUnit'] = md.image(series).Pixels.PhysicalSizeZUnit
    
    # get all image IDs
    for i in range(md.get_image_count()):
        metadata['ImageIDs'].append(i)

    # get information baout the instrument and objective
    metadata['InstrumentID'] = md.instrument(series).get_ID()
    metadata['DetectorModel'] = md.instrument(series).Detector.get_Model()
    metadata['DetectorID'] = md.instrument(series).Detector.get_ID()
    metadata['DetectorModel'] = md.instrument(series).Detector.get_Type()
    metadata['NA'] = md.instrument(series).Objective.get_LensNA()
    metadata['ObjID'] = md.instrument(series).Objective.get_ID()
    metadata['ObjMag'] = md.instrument(series).Objective.get_NominalMagnification()
    
    # get channel names
    for c in range(metadata['SizeC']):
        metadata['Channels'].append(md.image(series).Pixels.Channel(c).Name)
        
    return metadata

In [4]:
image_name = os.path.basename(filename_ometiff)

if filename_ometiff.lower().endswith('.ome.tiff') or filename_ometiff.lower().endswith('.ome.tif'):
    
    # Return value is an 5D Array of order (T, Z, C, X, Y)
    (array5d, omexml) = io.read_ometiff(filename_ometiff)
    
metadata = get_metadata(filename_ometiff, omexml, series=0)

In [5]:
array5d.shape

(1, 1, 3, 1104, 1376)

In [6]:
metadata

{'Directory': '/home/sebi06/testdata',
 'Filename': 'Osteosarcoma_01.ome.tiff',
 'Name': '',
 'AcqDate': '2018-06-07T21:29:00.9578115Z',
 'TotalSeries': 1,
 'SizeX': 1376,
 'SizeY': 1104,
 'SizeZ': 1,
 'SizeC': 3,
 'SizeT': 1,
 'DimOrder BF': 'XYZTC',
 'NA': '0.8',
 'ObjMag': '20',
 'ObjID': 'Objective:1',
 'XScale': 0.454,
 'YScale': 0.454,
 'ZScale': None,
 'XScaleUnit': None,
 'YScaleUnit': None,
 'ZScaleUnit': None,
 'DetectorModel': None,
 'DetectorName': [],
 'DetectorID': 'Detector:1',
 'InstrumentID': 'Instrument:1',
 'Channels': ['AF568_2', 'AF488', 'DAPI'],
 'ImageIDs': [0]}

In [7]:
import ipywidgets as widgets

t = widgets.IntSlider(description='T:',
                      min=1,
                      max=metadata['SizeT'],
                      step=1,
                      value=1,
                      continuous_update=False)

z = widgets.IntSlider(description='Z:',
                      min=1,
                      max=metadata['SizeZ'],
                      step=1,
                      value=1,
                      continuous_update=False)

c = widgets.IntSlider(description='C:',
                      min=1,
                      max=metadata['SizeC'],
                      step=1,
                      value=1)

r = widgets.IntRangeSlider(description='Display Range:',
                           min=array5d.min(),
                           max=array5d.max(),
                           step=1,
                           value=[array5d.min(), array5d.max()],
                           continuous_update=False)

ui = widgets.VBox([t, z, c, r])

In [8]:
def display_image(array5d, t=0, c=0, z=0, vmin=0, vmax=1000):
    image = array5d[t-1, z-1, c-1, :, :]
    # display the labelled image
    fig, ax = plt.subplots(figsize=(10, 10))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    im = ax.imshow(image, vmin=vmin, vmax=vmax, interpolation='nearest', cmap=cm.gray)
    fig.colorbar(im, cax=cax, orientation='vertical')
    print('Min-Max (Current Plane):', image.min(), '-', image.max())

def get_TZC(t, z, c, r):
    
    #minvalue = array5d[t-1, z-1, c-1, :, :].min()
    #maxvalue = array5d[t-1, z-1, c-1, :, :].max()
    #r.min = minvalue
    #r.max = maxvalue
    display_image(array5d, t=t, z=z, c=c, vmin=r[0], vmax=r[1])

# disable slider that are not needed
if metadata['SizeT'] == 1:
    t.disabled=True
if metadata['SizeZ'] == 1:
    z.disabled=True
if metadata['SizeC'] == 1:
    c.disabled=True

In [9]:
out = widgets.interactive_output(get_TZC, { 't': t, 'z': z, 'c': c, 'r':r})
display(ui, out)

Output()